## Homework II for ISDN6380B

Siyan HU, student no. 12217679

### 1. Load Dataset and Generate Labels

In [6]:
import sys
import os
import random

cwd = os.getcwd()
sys.path.append(cwd)

import file_io as fio

Load dataset

In [9]:
full_image_folder = fio.createPath(fio.sep, [cwd, 'data', 'images'])
full_image_paths = fio.traverse_dir(full_image_folder, full_path=True, towards_sub=False)
full_image_paths = fio.filter_ext(filepath_list=full_image_paths, filter_out_target=False, ext_set=fio.img_ext_set)

full_bbox_folder = fio.createPath(fio.sep, [cwd, 'data', 'txt'])
full_bbox_paths = fio.traverse_dir(full_bbox_folder, full_path=True, towards_sub=False)
full_bbox_paths = fio.filter_ext(filepath_list=full_bbox_paths, filter_out_target=False, ext_set=['txt'])

if (len(full_image_paths) == len(full_bbox_paths)) == False:
    print("ERROR: check files again or add a matching function.")

In [25]:
trainval_base = 1.0
trainval_lod = 0.9
train_lod = 0.9

total_sample_num = len(full_image_paths)
list_index = range(total_sample_num)

tv_num = round(total_sample_num * trainval_lod)
tr_num = round(tv_num * train_lod)

trainval_set = random.sample(list_index, tv_num)
train_set = random.sample(trainval_set , tr_num)
val_set = list(set(trainval_set).difference(set(train_set)))
test_set = list(set(list_index).difference(set(trainval_set)))

split_dir = fio.createPath(fio.sep, [cwd, 'data', 'dataset'])

for i in list_index:
    img_path = full_image_paths[i]
    if i in trainval_set :
        with open(split_dir + fio.sep + 'trainval.txt', 'a+') as file_trainval:
            file_trainval.write(img_path + '\n')
        if i in train_set :
            with open(split_dir + fio.sep + 'train.txt', 'a+') as file_train:
                file_train.write(img_path + '\n')
        else:
            with open(split_dir + fio.sep + 'val.txt', 'a+') as file_val:
                file_val.write(img_path + '\n')
    else:
        with open(split_dir + fio.sep + 'test.txt', 'a+') as file_test:
            file_test.write(img_path + '\n')

print("Spliting database: {} train+validation samples.\n {} are pure training data, \nand {} testing images".format(trainval_set, train_set, test_set))

Spliting database: [4, 38, 14, 35, 2, 27, 11, 1, 42, 18, 5, 43, 44, 32, 16, 28, 24, 10, 15, 49, 46, 13, 22, 0, 19, 34, 37, 31, 12, 23, 47, 45, 26, 21, 33, 30, 39, 25, 6, 36, 20, 40, 7, 9, 17, 8] train+validation samples.
 [16, 34, 0, 35, 31, 40, 14, 43, 7, 20, 47, 32, 27, 23, 36, 9, 37, 26, 25, 18, 12, 49, 42, 6, 19, 24, 39, 28, 21, 30, 22, 33, 8, 1, 38, 2, 4, 45, 5, 46, 10] are pure training data, 
and [3, 41, 48, 50, 29] testing images


create yaml

In [34]:
yaml_filepath = fio.createPath(fio.sep, [cwd, 'data'], 'chessboard.yaml')
with open(yaml_filepath, 'w', encoding='utf-8') as f:
    f.write('train: ' + split_dir + fio.sep + 'train.txt' + '\n')
    f.write('val: ' + split_dir + fio.sep + 'val.txt' + '\n')
    f.write('test: ' + split_dir + fio.sep + 'test.txt' + '\n\n')
    f.write('nc: 1\n')
    f.write('names:\n')
    f.write('  0: chessboard')

### 2. Install YOLOv8 and Import

conda install pytorch torchvision torchaudio pytorch-cuda=12.1 ultralytics -c pytorch -c nvidia -c conda-forge

In [41]:
from ultralytics import YOLO
import time

### 3. Train YOLOv8

In [40]:
def current_timestamp(micro_second=False):
    t = time.time()
    if micro_second:
        return int(t * 1000 * 1000)
    else:
        return int(t * 1000)

In [45]:
model = YOLO("yolov8n.yaml").load("yolov8n.pt")

save_path = fio.createPath(fio.sep, [cwd, 'data', str(current_timestamp(False))])
fio.ensure_dir(save_path)
result = model.train(data=yaml_filepath, epochs=6, device=[0], save_dir=save_path)

Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/chessboard.yaml, epochs=6, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_

train: Scanning /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels... 0 images, 41 backgrounds, 0 corrupt: 100%|██████████| 41/41 [00:00<00

train: WARNING ⚠️ No labels found in /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels.cache. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.
train: New cache created: /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels.cache
WARNING ⚠️ No labels found in /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.



val: Scanning /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels... 0 images, 5 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<00:00, 

val: WARNING ⚠️ No labels found in /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels.cache. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.
val: New cache created: /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels.cache
WARNING ⚠️ No labels found in /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.


...

                   all          5          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels



6 epochs completed in 0.034 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 6.2MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 113.18it/s]

                   all          5          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train4


### 4. Visualise the training result

In [46]:
metrics = model.val()
metrics.box.map

Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients


val: Scanning /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels.cache... 0 images, 5 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?

WARNING ⚠️ No labels found in /home/siyanlinux/Documents/ISDN6380B_Homework/hw2/data/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.18it/s]

                   all          5          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


Speed: 0.1ms preprocess, 21.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train42


0.0

### 5. Prediction

In [48]:
predict_image_folder = fio.createPath(fio.sep, [cwd, 'data_predict', 'Chessboard_test1'])
predict_image_paths = fio.traverse_dir(predict_image_folder, full_path=True, towards_sub=False)
predict_image_paths = fio.filter_ext(filepath_list=predict_image_paths, filter_out_target=False, ext_set=fio.img_ext_set)

In [50]:
predictions = model(predict_image_paths)
probs_save_dir = fio.createPath(fio.sep, [save_path], 'predictions')
fio.ensure_dir(probs_save_dir)

index = 0
for pred in predictions:
    box = pred.boxes
    masks = pred.masks
    keypoints = pred.keypoints
    probs = pred.probs

    original_image_path = predict_image_paths[index]
    original_image_name = fio.get_filename_components(original_image_path)[1]
    probs_save_path = fio.createPath(fio.sep, [save_path, 'predictions'], str(original_image_name) + '.jpg')
    pred.save(probs_save_path)
    index += 1


0: 640x640 (no detections), 0.6ms
1: 640x640 (no detections), 0.6ms
2: 640x640 (no detections), 0.6ms
3: 640x640 (no detections), 0.6ms
4: 640x640 (no detections), 0.6ms
5: 640x640 (no detections), 0.6ms
6: 640x640 (no detections), 0.6ms
7: 640x640 (no detections), 0.6ms
8: 640x640 (no detections), 0.6ms
9: 640x640 (no detections), 0.6ms
10: 640x640 (no detections), 0.6ms
11: 640x640 (no detections), 0.6ms
12: 640x640 (no detections), 0.6ms
13: 640x640 (no detections), 0.6ms
14: 640x640 (no detections), 0.6ms
15: 640x640 (no detections), 0.6ms
16: 640x640 (no detections), 0.6ms
17: 640x640 (no detections), 0.6ms
18: 640x640 (no detections), 0.6ms
19: 640x640 (no detections), 0.6ms
20: 640x640 (no detections), 0.6ms
21: 640x640 (no detections), 0.6ms
22: 640x640 (no detections), 0.6ms
23: 640x640 (no detections), 0.6ms
24: 640x640 (no detections), 0.6ms
25: 640x640 (no detections), 0.6ms
26: 640x640 (no detections), 0.6ms
27: 640x640 (no detections), 0.6ms
28: 640x640 (no detections), 

IndexError: list index out of range